In [95]:
import fsps
from matplotlib import pylab as plt
import numpy as np
from scipy.interpolate import interp1d
np.set_printoptions(threshold=np.nan)
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt
from math import *
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord,Angle
from astropy.cosmology import FlatLambdaCDM



sed1 = np.loadtxt("wave_spec.txt")
filtertranf435w = np.loadtxt("filter_wfc_acs_f435w.txt")
filtertranf606w = np.loadtxt("filter_wfc_acs_f606w.txt")
filtertranf775w = np.loadtxt("filter_wfc_acs_f775w.txt")
filtertranf850lp = np.loadtxt("filter_wfc_acs_f850lp.txt")
filtertranf105w = np.loadtxt("filter_wfc3_ir_f105w.txt")
filtertranf160w = np.loadtxt("filter_wfc3_ir_f160w.txt")
filtertranf200w = np.loadtxt("filter_jwst_f200w.txt")
filtertranf277w = np.loadtxt("filter_jwst_f277w.txt")
filtertranf356w = np.loadtxt("filter_jwst_f356w.txt")
filtertranf444w = np.loadtxt("filter_jwst_f444w.txt")



wavecut = sed1[:,0]
sedcut = sed1[:,1]
filterwave = filtertran[:,0]
filterspec = filtertran[:,1]
#sedcut.astype(float)
#mean = np.mean(sedcut)
#print (mean)
#print (type(wavecut))
print (type(wavecut))

<class 'numpy.ndarray'>


In [98]:
class lambdafunc(object):
    def update(self,wavelength,value,units):
        if not len(wavelength)==len(value):
            raise ValueError('The length of wavelength list does not equal to the length of value list')
        else:
            self.wavelength=np.array(wavelength)*float(1*units[0].cgs/u.Angstrom)
            self.value=np.array(value)
            self.units=[u.Angstrom.cgs,units[1].cgs]
            self.interval=(np.append(self.wavelength,0)-np.append(0,self.wavelength))[1:-1]
            self.midvalue=(np.append(self.value,0)+np.append(0,self.value))[1:-1]/2.0



    def getvalue(self,x):
        return np.interp(x,self.wavelength,self.value)

    def plot(self):
        plt.plot(self.wavelength,self.value)
        plt.xlabel('Wavelength / $\AA$',fontsize=16)
        plt.ylabel('Arbitrary Unit',fontsize=16)
        plt.show()


class FiltCurve(lambdafunc):
    def __init__(self,wavelength,value,units=[u.Angstrom,u.Quantity(1)]):
        self.update(wavelength,value,units)


class Spectrum(lambdafunc):
    def __init__(self,wavelength,value,units=[u.Angstrom,u.erg/u.s/u.cm/u.cm/u.Angstrom],mode='OBS'):
        self.update(wavelength,value,units)
        self.mode=mode

    def flux(self,filt):
        newflux=self.getvalue(np.array(filt.wavelength*filt.units[0]/self.units[0]))
        filtered_flux=newflux*filt.value


        midflux=(np.append(filtered_flux,0)+np.append(0,filtered_flux))[1:-1]/2.0
        interval=np.abs(filt.interval)

        return np.sum(interval*midflux)*filt.units[0]*self.units[1]

    def magnitude(self,filt,style='AB'):
        '''
        Calculate the magnitude in the input filter.
        '''
        ### Now, AB mag only ###

        objflux=self.flux(filt)
        stdspec=((3631*u.Jy).cgs*3e10*u.cm/u.s/((filt.wavelength*filt.units[0]).cgs)**2).to(self.units[1])

        midstdspec=(np.append(stdspec,0)+np.append(0,stdspec))[1:-1]/2.0

        interval=np.abs(filt.interval)
        stdflux=np.sum(interval*midstdspec*filt.midvalue)*filt.units[0]*self.units[1]

        mag=-2.5*np.log10(float(objflux/stdflux))
        return mag

    

        self.mode='ABS'

   

    def normalize(self,filt,mag,style='AB'):
        mag0=self.magnitude(filt,style)
        scale=10**(0.4*(mag0-mag))

        self.value=self.value*scale

def main():
    '''
    An example.
    '''

    '''
    Always follow:

    Wavelength in angtrom.
    Flux in erg s-1 cm-2 A-1
    '''
    

    #input the filter
    Filter = FiltCurve(filtertranf160w[:,0],filtertranf160w[:,1])
  
    #input the final spec and observed wavelength in angstroms
    spec = Spectrum(wavecut, sedcut)
    
    mag = spec.magnitude(Filter)
    print(mag)

if __name__=='__main__':
    main()


25.4170574681
